In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from advi.model import ADVIModel
from models.ard import Ard
from models.ard import sep_params
#from models.ard import joint_log_prob, return_initial_state, sep_params, log_likelihood, some_kind_of_loss
# NEXT TO DO MUST CHANGE INITAL CHAIN STATES TO SAMPLE FROM DISTRIBUTION
# Target distribution is proportional to: `exp(-x (1 + x))`.

# Initialize the HMC transition kernel.



/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Making training data

In [2]:
def make_training_data(num_samples, dims, sigma):
  """
  Creates training data when half of the regressors are 0
  """
  x = np.random.randn(dims, num_samples).astype(np.float64)
  w = sigma * np.random.randn(1, dims).astype(np.float64)
  noise = np.random.randn(num_samples).astype(np.float64)
  w[:,:int(dims/2)] = 0
  y = w.dot(x) + noise
    
  return y, x, w

def sep_training_test(y,x,test):
  y_train = y[:,test:]
  x_train = x[:,test:]
  
  y_test = y[:,:test]
  x_test = x[:,:test]
  return y_train, y_test, x_train, x_test
num_features = 250

y, x, w = make_training_data(100, num_features, 2)
y_train, y_test, x_train, x_test = sep_training_test(y,x,100)
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)
print(x_test.shape)



(1, 0)
(1, 100)
(250, 0)
(250, 100)


## Useful functions when using HMC 

In [3]:
def state_to_log_pred(states, y_test, x_test):
    sample_mean = tf.reduce_mean(states, axis=[0])
    return model.log_likelihood(y_test, x_test, states)

def state_to_loss(states, y_test, x_test):
    sample_mean = tf.reduce_mean(states, axis=[0])
    w, _, _ = sep_params(states, num_features)
    return model.some_kind_of_loss(y_test, x_test, w)

## Defining summary writer

In [4]:
summary_writer = tf.compat.v2.summary.create_file_writer('/tmp/summary_chain', flush_millis=10000)
def trace_fn(state, results):
  with tf.compat.v2.summary.record_if(tf.equal(results.step % 10, 0)):
    tf.compat.v2.summary.scalar("log pred", state_to_log_pred(state, y_test, x_test), step=tf.cast(results.step, tf.int64))
    return ()

In [5]:
model = Ard(num_features)
initial_chain_state = model.return_initial_state()
wstart, tau, alpha = sep_params(initial_chain_state, num_features)

## Defining kernels for HMC and NUTs

In [6]:
num_results = int(10000)
num_burnin_steps = int(800)
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=model.joint_log_prob,
        num_leapfrog_steps=3,
        step_size=0.1),
    num_adaptation_steps=int(num_burnin_steps * 0.8))

nuts = tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn=model.joint_log_prob, step_size=1., max_tree_depth=10, max_energy_diff=1000.0,
    unrolled_leapfrog_steps=1, parallel_iterations=10, seed=None, name=None)

In [7]:
def run_chain():
  print("running chain")
  # Run the chain (with burn-in).
  states, is_accepted = tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_chain_state,
      kernel=adaptive_hmc,
    trace_fn=trace_fn)
  return states, is_accepted

    

In [8]:
def run_chain_nuts():
  print("running chain")
  # Run the chain (with burn-in).
  states, is_accepted = tfp.mcmc.sample_chain(
      num_results=20,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_chain_state,
      kernel=nuts)
  return states, is_accepted
    

## ADVI Stuff starts here

### Functions to pass in to summary writer

In [9]:
def get_ll_from_advi(y_test, x_test, advi, regression_model):
    theta_intermediate = advi.sample()
    theta_intermediate = tf.squeeze(theta_intermediate)
    likelihood = regression_model.log_likelihood(y_test, x_test, theta_intermediate)
    return likelihood

def get_loss_from_advi(y_test, x_test, advi, regression_model):
    theta_intermediate = advi.sample()
    theta_intermediate = tf.squeeze(theta_intermediate)
    w, _, _ = sep_params(theta_intermediate, num_features)
    return regression_model.some_kind_of_loss(y_test, x_test, w)

In [ ]:
def run_advi(nsteps, step_size, dim, log_prob, bijector, m, regression_model):
        advi = ADVIModel(dim, log_prob, bijector, m)
        for t in range(nsteps):
            grad_mu, grad_omega = advi.gradients()
            advi.mu = tf.add(advi.mu, step_size * grad_mu)
            advi.omega = tf.add(advi.omega, step_size * grad_omega)
            tf.summary.scalar('elbo', advi.elbo(), step=t)
            tf.summary.scalar('log likelihood', get_ll_from_advi(y_test, x_test, advi, regression_model), step=t)
            tf.summary.scalar('loss', get_loss_from_advi(y_test, x_test, advi, regression_model), step=t)
            if(t%1 == 0):
                theta_intermediate = advi.sample()
                theta_intermediate = tf.squeeze(theta_intermediate)
                wpred, _, _ = sep_params(theta_intermediate, num_features)
                loss = regression_model.some_kind_of_loss(y_test, x_test, wpred)
                likelihood = regression_model.log_likelihood(y_test, x_test, theta_intermediate)
                elbo_loss = advi.elbo()
                print(loss)
                print("elbo", elbo_loss)
        theta = advi.sample()
        return theta

In [ ]:
# looking at starting weights
start_w, _, _ = sep_params(initial_chain_state, num_features)
# looking at starting model
start_likelihood = model.log_likelihood(y_test, x_test, initial_chain_state)
print("start likelihood", start_likelihood)
# target function 
joint_log_prob2 = lambda *args: model.joint_log_prob(y_train, x_train, *args)
bij = tfp.bijectors.Log()
num_results = 10000
step_size = 0.001
# run the advi
with summary_writer.as_default():
    theta = run_advi(num_results,
                     step_size,
                     model.num_params,
                     joint_log_prob2,
                     bij,
                     10, model)


('start likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-18766.503086894638>)
tf.Tensor(5081.81700426972, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1765.1893192715788>)
tf.Tensor(3688.803462333918, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1589.7554702404996>)
tf.Tensor(3618.2215646902464, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1482.2189695980437>)
tf.Tensor(3849.8160960746955, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1422.6827119819106>)
tf.Tensor(3491.836383271302, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1387.9205426243077>)
tf.Tensor(3883.2313717691673, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1343.7196831155813>)
tf.Tensor(3590.0191170226044, shape=(), dtype=float64)
('elbo', <tf.Tensor: shape=(), dtype=float64, numpy=-1312.6242808661978>)
tf.Tensor(3774.

In [ ]:
start_loss = state_to_loss(initial_chain_state, y_test, x_test)
print("start loss", start_loss)
print("theta", theta.shape)
theta = tf.squeeze(theta)
w, _, _ = sep_params(theta, num_features)
loss = model.some_kind_of_loss(y_test, x_test, w)

print("loss", loss)



In [ ]:
initial_chain_state = return_initial_state(num_features)
sample_mean = tf.reduce_mean(states, axis=[0])
wend, alpha, tau = sep_params(sample_mean, num_features)

print("start", wstart)
loss_start = some_kind_of_loss(wstarty_test, x_test, )
print("loss start", loss_start)

print("actual",w)
print("end", wend)
print("start", wstart)
loss_end = some_kind_of_loss(wend,y_test, x_test, )
print("loss start", loss_start)
print("loss end", loss_end)


In [ ]:
sample_mean = tf.reduce_mean(states, axis=[0])
wend, alpha, tau = sep_params(sample_mean, num_features)

